In [1]:
import pandas as pd

super_dataset = pd.read_csv('../../../data/stage_I/superDataset.csv')
world_dataset = pd.read_csv('../../../data/stage_II/owid-covid-data.csv')

countries = ['Indonesia', 'Pakistan', 'Nigeria', 'Brazil', 'Bangladesh', 'United States']

In [2]:
filtered_dataset = world_dataset[world_dataset.location.isin(countries)]
filtered_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,...,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.80,72.59,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86,0.926


In [22]:
us_dataset = filtered_dataset[filtered_dataset.location == 'United States'].fillna(0)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,...,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926


In [45]:
us_dataset['date'] = pd.to_datetime(us_dataset.date)
us_dataset['week'] = us_dataset.date.apply(lambda x: str(x.year) + "_" + str(x.weekofyear))
us_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,week
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_8
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2021_9


In [46]:
from datetime import datetime

In [50]:
us_dataset.groupby('week').mean().new_cases.apply(lambda x: math.ceil(x))

week
2020_10        70
2020_11       385
2020_12      4527
2020_13     15574
2020_14     29636
2020_15     31441
2020_16     28321
2020_17     29629
2020_18     27216
2020_19     24490
2020_20     22388
2020_21     22385
2020_22     20898
2020_23     20701
2020_24     21326
2020_25     26864
2020_26     39062
2020_27     49136
2020_28     58974
2020_29     66582
2020_30     65515
2020_31     61834
2020_32     53887
2020_33     50607
2020_34     42686
2020_35     41603
2020_36     40981
2020_37     34825
2020_38     41057
2020_39     43991
2020_4          1
2020_40     42923
2020_41     50138
2020_42     56196
2020_43     70767
2020_44     86758
2020_45    116507
2020_46    150748
2020_47    173370
2020_48    163886
2020_49    200084
2020_5          1
2020_50    216106
2020_51    216737
2020_52    185330
2020_53    209753
2020_6          1
2020_7          1
2020_8          1
2020_9          3
2021_1     245104
2021_2     217701
2021_3     169323
2021_4     148427
2021_5     117222
2021_

In [48]:
pd.DataFrame(us_dataset.groupby('week').mean().new_cases.apply(lambda x: math.ceil(x))).reset_index().week.apply(lambda x: datetime.strptime(x + '_1', '%G_%V_%u'))

0    2020-03-02
1    2020-03-09
2    2020-03-16
3    2020-03-23
4    2020-03-30
5    2020-04-06
6    2020-04-13
7    2020-04-20
8    2020-04-27
9    2020-05-04
10   2020-05-11
11   2020-05-18
12   2020-05-25
13   2020-06-01
14   2020-06-08
15   2020-06-15
16   2020-06-22
17   2020-06-29
18   2020-07-06
19   2020-07-13
20   2020-07-20
21   2020-07-27
22   2020-08-03
23   2020-08-10
24   2020-08-17
25   2020-08-24
26   2020-08-31
27   2020-09-07
28   2020-09-14
29   2020-09-21
30   2020-01-20
31   2020-09-28
32   2020-10-05
33   2020-10-12
34   2020-10-19
35   2020-10-26
36   2020-11-02
37   2020-11-09
38   2020-11-16
39   2020-11-23
40   2020-11-30
41   2020-01-27
42   2020-12-07
43   2020-12-14
44   2020-12-21
45   2020-12-28
46   2020-02-03
47   2020-02-10
48   2020-02-17
49   2020-02-24
50   2021-01-04
51   2021-01-11
52   2021-01-18
53   2021-01-25
54   2021-02-01
55   2022-01-03
56   2021-02-08
57   2021-02-15
58   2021-02-22
59   2021-03-01
Name: week, dtype: datetime64[ns]

In [ ]:
datetime.datetime.strptime(d + '-1', '%G-W%V-%u')

In [28]:
us_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,week
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.000,0.0,0.0,0.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,8_2021
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021


In [26]:
import math

pd.merge(us_dataset, pd.DataFrame(us_dataset.groupby('week').mean().new_cases.apply(lambda x: math.ceil(x))), on="week")

,iso_code,continent,location,date,total_cases,new_cases_x,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,week,new_cases_y
0,USA,North America,United States,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,1
1,USA,North America,United States,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,1
2,USA,North America,United States,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,1
3,USA,North America,United States,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,1
4,USA,North America,United States,2020-01-26,5.0,3.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,8_2021,67314
404,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,62149
405,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,62149
406,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,62149


In [16]:
us_dataset

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,week,weekly_mean
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,NaN
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,NaN
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,NaN
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,NaN
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.000,0.0,0.0,0.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,4_2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69377,USA,North America,United States,2021-02-28,28578548.0,51204.0,67314.000,512545.0,1037.0,1943.857,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,8_2021,NaN
69378,USA,North America,United States,2021-03-01,28637313.0,58765.0,67752.429,514061.0,1516.0,1978.857,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,NaN
69379,USA,North America,United States,2021-03-02,28694071.0,56758.0,65630.714,515985.0,1924.0,1932.000,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,NaN
69380,USA,North America,United States,2021-03-03,28759980.0,65909.0,64413.571,518453.0,2468.0,1835.143,...,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,9_2021,NaN
